In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


In [11]:
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] 

# Print your answer
print(list_of_tables)


['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


In [14]:

def get_table(table_name, max_results=10000):
    # Construct a reference to the "posts_answers" table
    answers_table_ref = dataset_ref.table(table_name)

    # API request - fetch the table
    answers_table = client.get_table(answers_table_ref)

    # Preview the first five lines of the "posts_answers" table
    return client.list_rows(answers_table, max_results=max_results).to_dataframe()

get_table('users', 3)

,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,261,Mattias,None,None,2008-08-04 08:51:21.440000+00:00,2018-11-13 15:49:56.143000+00:00,United Kingdom,4661,24,0,265,None,http://www.bothernomore.com/
1,999,BlaM,"<p>Professional Web Developer since 2001, amat...",None,2008-08-11 12:32:21.780000+00:00,2020-08-04 11:02:46.297000+00:00,"Thomas-Jefferson-Straße, Mannheim, Deutschland",26102,1163,99,1513,None,http://blog.deobald.org
2,1555,alex77,Student,None,2008-08-16 16:22:46.600000+00:00,2009-03-24 16:50:32.570000+00:00,United Kingdom,538,50,1,106,None,http://Nope


In [24]:
def get_all_users(location=None):
    if location:
        questions_query = f"SELECT id, display_name,reputation, up_votes, down_votes FROM `bigquery-public-data.stackoverflow.users` WHERE location='{location}' "
    else:
        questions_query = f"SELECT id, display_name,reputation, up_votes, down_votes FROM `bigquery-public-data.stackoverflow.users`"

    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 1 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    questions_query_job = client.query(questions_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    questions_results = questions_query_job.to_dataframe()

    return questions_results

get_all_users('United States')

      id    display_name  reputation  up_votes  down_votes
0   3460           Pauly         158        26           0
1   3578        user3578         103         0           0
2   5462     Dean Poulin        1065        26           5
3  11196   Jim Zimmerman        1777        13           0
4  24875  patricksweeney        3369       278        1610


,id,display_name,reputation,up_votes,down_votes
0,3460,Pauly,158,26,0
1,3578,user3578,103,0,0
2,5462,Dean Poulin,1065,26,5
3,11196,Jim Zimmerman,1777,13,0
4,24875,patricksweeney,3369,278,1610
...,...,...,...,...,...
21204,1886417,Jonny5 Alive,41,6,0
21205,8459100,Jerry Liu,41,0,0
21206,7382452,Jordan Negative,41,1,0
21207,6056224,Alireza Ebrahimvandi,41,34,0


In [46]:
def get_all_tags():
    questions_query = f"SELECT id, tag_name FROM `bigquery-public-data.stackoverflow.tags`"

    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 1 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    questions_query_job = client.query(questions_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    questions_results = questions_query_job.to_dataframe()
    
    # return a list of tags
    return list(questions_results.iloc[:,1])

get_all_tags()

['vml',
 'integrity',
 'icollection',
 'round-robin',
 'jlayeredpane',
 'relevance',
 'dxf',
 'ant-contrib',
 'preloading',
 'libpq',
 'sqlyog',
 'applescript-objc',
 'scalacheck',
 'fgetc',
 'name-lookup',
 'eclipse-gef',
 'backbone-relational',
 'ubuntu-11.10',
 'genfromtxt',
 'docpad',
 'accumulo',
 'openfl',
 'akka-persistence',
 'location-services',
 'hexo',
 'pysftp',
 'connected-components',
 'anko',
 'mixed-integer-programming',
 'mac-catalyst',
 'jeditable',
 'plaintext',
 'ps',
 'terrain',
 'npoi',
 'doctrine-odm',
 'apache-commons-httpclient',
 'size-classes',
 'android-database',
 'gs-vlookup',
 'flat-file',
 'curve',
 'confluence',
 'text-parsing',
 'listviewitem',
 'same-origin-policy',
 'custom-taxonomy',
 'aiohttp',
 'django-class-based-views',
 'akka-http',
 'uidatepicker',
 'jlist',
 'pyserial',
 'google-adwords',
 'android-styles',
 'viewport',
 'psycopg2',
 'angular-routing',
 'mvvmcross',
 'es6-promise',
 'settimeout',
 'pytest',
 'pyqt4',
 'bluetooth-lowenergy',
 

In [30]:
def get_questions_by_user(user_id):
    questions_query = f"SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` \
                        WHERE owner_user_id={user_id}"
    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 1 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**11)
    questions_query_job = client.query(questions_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    questions_results = questions_query_job.to_dataframe()

    return questions_results

get_questions_by_user(13048023)

         id                                              title  \
0  60645095  Unable to successfully upload a file via Slack...   

                                                body accepted_answer_id  \
0  <p>I'm unable to successfully upload a file ev...               None   

   answer_count  comment_count community_owned_date  \
0             2              0                 None   

                     creation_date favorite_count  \
0 2020-03-11 22:14:06.287000+00:00           None   

                last_activity_date                   last_edit_date  \
0 2020-03-26 04:50:05.180000+00:00 2020-03-12 13:00:13.987000+00:00   

  last_editor_display_name  last_editor_user_id owner_display_name  \
0                     None             13048023               None   

   owner_user_id parent_id  post_type_id  score   tags  view_count  
0       13048023      None             1      1  slack         256  


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,60645095,Unable to successfully upload a file via Slack...,<p>I'm unable to successfully upload a file ev...,None,2,0,None,2020-03-11 22:14:06.287000+00:00,None,2020-03-26 04:50:05.180000+00:00,2020-03-12 13:00:13.987000+00:00,None,13048023,None,13048023,None,1,1,slack,256


In [49]:
def get_answered_questions_by_user(user_id):
    questions_query = f"SELECT a.id, a.body, a.owner_user_id AS uid, q.id AS qid \
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q \
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a \
                    ON q.id = a.parent_id \
                WHERE a.owner_user_id = {user_id}"
    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 1 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**11)
    questions_query_job = client.query(questions_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    questions_results = questions_query_job.to_dataframe()

    # Preview results
    print(questions_results.head())
    return questions_results

"""
User 11486952 - there are answers but no questions
User 10904707 - there are questions but no answers
User 10977933 - there are both answers and questions
User 11040013 - there are no answers and no questions
"""
get_answered_questions_by_user('11486952')
    

         id                                               body   user_id  \
0  56123675  <p>Alpha and beta are both hyper parameters an...  11486952   

        qid  
0  56123272  


,id,body,user_id,qid
0,56123675,<p>Alpha and beta are both hyper parameters an...,11486952,56123272


In [43]:
def get_tag_by_question(qid):
    questions_query = f"SELECT tags \
                FROM `bigquery-public-data.stackoverflow.posts_questions` \
                WHERE id = {qid}"
    # Set up the query (cancel the query if it would use too much of 
    # your quota, with the limit set to 1 GB)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**11)
    questions_query_job = client.query(questions_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    questions_results = questions_query_job.to_dataframe()
    
    res = questions_results.iloc[0,0].split('|')
    return res

get_tag_by_question(10880947)

['visual-studio', 'msbuild', 'msbuild-task', 'msbuild-4.0']